<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

176. Second Highest Salary

https://leetcode.com/problems/second-highest-salary/description/

```sql
SELECT max(salary) as SecondHighestSalary
FROM (
SELECT
    id, salary,
    dense_rank() OVER (ORDER BY salary DESC) AS rn
FROM Employee e)
WHERE rn = 2
```

In [4]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [31]:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == 2")[['salary']].rename({"salary":"SecondHighestSalary"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{"SecondHighestSalary": None}])

177. Nth Highest Salary

https://leetcode.com/problems/nth-highest-salary/description/

```sql
CREATE OR REPLACE FUNCTION NthHighestSalary(N INT) RETURNS TABLE (Salary INT) AS $$
BEGIN
  RETURN QUERY (
    -- Write your PostgreSQL query statement below.
    SELECT max(sub.salary) FROM (
    SELECT
        e.salary,
        dense_rank() OVER (ORDER BY e.salary DESC) as rn
    FROM Employee e) sub
    WHERE sub.rn = N
      
  );
END;
$$ LANGUAGE plpgsql;
```

In [32]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['Id', 'Salary']).astype({'Id':'Int64', 'Salary':'Int64'})

In [34]:
def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == @N")[['salary']].rename({"salary":f"getNthHighestSalary({N})"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{f"getNthHighestSalary({N})": None}])

178. Rank Scores

https://leetcode.com/problems/rank-scores/description/

```sql
SELECT
    score,
    dense_rank() OVER (ORDER BY score DESC) as rank
FROM Scores
ORDER BY score DESC
```

In [35]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})

In [41]:
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    return scores.assign(rank = scores.score.rank(method='dense', ascending=False))[['score','rank']].sort_values(by='score', ascending=False)